In [1]:
import os
import pymysql
from dotenv import load_dotenv
load_dotenv()

HOST = os.getenv('HOST')
USER = os.getenv('MYSQL_USER')
PASSWORD = os.getenv('MYSQL_ROOT_PASSWORD')
DB = os.getenv('MYSQL_DATABASE')

connection = pymysql.connect(host=HOST, 
                      user = USER,
                     password = PASSWORD,
                     db = DB)

cursor = connection.cursor()

## Attribute Selection

In [51]:
cursor.execute("SELECT * FROM pedidos")
cursor.fetchall()[0]

"""
Caption:
numero
data_pedido
hora_pedido
cliente
endereco
telefone
tipo_entrega
valor_pizza
valor_borda
valor_refrigerante
valor_entrga
valor_total
hora_entrega
tempo
"""

(1,
 '2011-10-07',
 '18:45:00',
 'Cliente 1',
 'Endereco 1',
 '00-000-0000',
 'Buscar',
 32.0,
 0.0,
 0.0,
 0.0,
 32.0,
 '19:15:00',
 '00:30:00')

In [46]:
#Defining the columns that are going to be droped
numero = "ALTER TABLE pedidos DROP COLUMN numero;"
cliente = "ALTER TABLE pedidos DROP COLUMN cliente"
endereco = "ALTER TABLE pedidos DROP COLUMN endereco"
telefone = "ALTER TABLE pedidos DROP COLUMN telefone"
valor_pizza = "ALTER TABLE pedidos DROP COLUMN valor_pizza"
valor_entrega = "ALTER TABLE pedidos DROP COLUMN valor_entrega"
hora_entrega = "ALTER TABLE pedidos DROP COLUMN hora_entrega"

In [47]:
lista = [numero, cliente, endereco, telefone,
        valor_pizza, valor_entrega, 
        hora_entrega]

In [55]:
for i in range(len(lista)):
    cursor.execute(lista[i])
connection.commit()

# IMPORTANT
>The process shown above can't be done in sqlite, because there are some limitations of sqlite.

In [59]:
#The extremes date
cursor.execute("SELECT MAX(data_pedido), MIN(data_pedido) FROM pedidos")
cursor.fetchall()

[('2015-02-03', '2011-10-07')]

* __I am going to use only the values where data_pedido < 2015__

In [62]:
cursor.execute("SELECT dayname(data_pedido) FROM pedidos")

OperationalError: no such function: dayname